<a href="https://colab.research.google.com/github/naok-000/lecture-ai-engineering-day3/blob/main/ai_engineering_03_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIエンジニアリング実践講座 第3回課題

- 講義で学んだRAG(Retrieval-Augmented Generation)技術を用いて、LLMの生成内容を改善する実践的な取り組みを行った．演習で利用したコードをベースに、独自の質問と参照文書を用いて実験を行い、RAGの効果を定量的・定性的に評価した．
- プログラムはGoogle Colab（無料版）T4 GPU を利用して実行した．

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/naok-000/lecture-ai-engineering-day3.git

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

## 1. ベースラインモデル評価

まずはベースモデルがどの程度知識を持っているか確かめる

In [ ]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
questions = [
    "what are the main vulnerabilities of the OBD-II port that make it a critical entry point for vehicle malware attacks?",
    "OBD-IIポートが車両のマルウェア攻撃における重要な侵入口となる主な脆弱性は何ですか？",
    "How do different wireless technologies (such as Bluetooth, Wi-Fi, Cellular, and DSRC) collectively contribute to the increased malware attack surface in modern intelligent vehicles?",
    "Bluetooth、Wi-Fi、セルラー、DSRC などの無線通信技術は、現代のインテリジェント車両におけるマルウェア攻撃の対象領域をどのようにして拡大しているのですか？"
]

for question in questions:
    print(f"question: {question}")
    response = generate_output(question)
    print(f"response: \n{response}")

In [ ]:
questions = [
    "What is the primary benefit of deploying Fixed Sensor Nodes (FSNs) in Vehicle-to-Infrastructure (V2I) cooperative systems, especially in complex urban scenarios?",
    "固定センサノード（FSN）をVehicle-to-Infrastructure（V2I）協調システムに導入する主な利点は何ですか？特に複雑な都市環境においては？",
    "How do cooperative object detection and cooperative tracking complement each other in enhancing safety and reliability in V2I autonomous driving systems, and what specific mechanisms enable this synergy?",
    "協調型物体検出と協調型追跡は、V2I型自動運転システムにおける安全性と信頼性の向上にどのように相互補完的に働きますか？また、この相乗効果を可能にする具体的なメカニズムは何ですか？",
    "Who is the author of 'Infrastructure Assisted Autonomous Driving: Research, Challenges, and Opportunities'?",
    "'Infrastructure Assisted Autonomous Driving: Research, Challenges, and Opportunities'の著者は誰ですか？"
]

for question in questions:
    print(f"question: {question}")
    response = generate_output(question)
    print(f"response: \n{response}")

## 2. RAGの活用

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

資料をpdfからテキストに変換する

参考：https://zenn.dev/quiver/articles/21c2978cf869db


In [ ]:
!pip install -U pypdfium2
import pypdfium2 as pdfium

def pdf_to_text(pdf):
    all_text = ""
    for page in pdf:
        textpage = page.get_textpage()
        text = textpage.get_text_range()
        all_text += text
    return all_text

pdf = pdfium.PdfDocument('/content/lecture-ai-engineering-day3/data/Infrastructure_Assisted_Autonomous_Driving_Research_Challenges_and_Opportunities.pdf')

raw_writedown = pdf_to_text(pdf)

documents = [text.strip() for text in raw_writedown.split(".")]

print(documents)

ドキュメントからLLMに渡す参考資料を抽出する

In [ ]:
# ドキュメントから質問に関連するものを抽出し，参考資料を作成
def generate_ref(question, documents):
  print(f"question: {question}\n")
  # Retrievalの実行
  query_embeddings = emb_model.encode([question], prompt_name="query")
  document_embeddings = emb_model.encode(documents)

  # 各ドキュメントの類似度スコア
  scores = (query_embeddings @ document_embeddings.T) * 100
  print(scores.tolist())

  topk = 5
  references = []
  for ref in [".".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

    query = f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] {question}"
    response = generate_output(query)

    print("\n\n対象となるドキュメント:\n", ref.replace(".", ".\n"))
    print("\n関連しているかどうか: ", response)

    if "yes" in response.lower():
      references.append(ref)
  return references

In [ ]:
references_list = []
for question in questions:
  references_list.append(generate_ref(question, documents))

print (references_list)

参考資料を渡して質問に回答させる

In [ ]:
for i in range(len(questions)):
  print(f"question: {questions[i]}")
  query =  f"[参考資料]\n{references_list[i]}\n\n[質問] {questions[i]}"
  response = generate_output(query)
  print(f"response: \n{response}")